In [ ]:
# Cell 1: imports and helper function
import os
import csv
import numpy as np
from scipy.io import loadmat
from PIL import Image

def export_mat_as_png_and_csv(mat_path, out_dir, csv_path, img_var, angle_var):
    data = loadmat(mat_path)
    X = data[img_var]                # H×W×1×N
    angles = data[angle_var].ravel() # (N,)
    N = X.shape[3]

    os.makedirs(out_dir, exist_ok=True)

    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "angle"])
        for i in range(N):
            # extract image and force uint8
            img_arr = X[:, :, 0, i]
            # if float, assume it's already 0–255 or 0–1; scale/clamp then cast
            img_arr = np.clip(img_arr, 0, 255)
            img_uint8 = img_arr.astype(np.uint8)
            # save PNG
            fname = f"{i:05d}.png"
            Image.fromarray(img_uint8).save(os.path.join(out_dir, fname))
            # write label
            writer.writerow([fname, float(angles[i])])


In [5]:
# Cell 2: export train & test sets
export_mat_as_png_and_csv(
    mat_path="DigitsDataTrain.mat",
    out_dir="train_images",
    csv_path="train_labels.csv",
    img_var="XTrain",
    angle_var="anglesTrain"
)

export_mat_as_png_and_csv(
    mat_path="DigitsDataTest.mat",
    out_dir="test_images",
    csv_path="test_labels.csv",
    img_var="XTest",
    angle_var="anglesTest"
)

print("Export complete: train_images/, test_images/, train_labels.csv, test_labels.csv")


OSError: cannot write mode F as PNG